# lab 1 | Regressions | Solutions

For your very first exercise in this course, you will try to predict the grades of students in a Portuguese school. For this, we will use (simplified) questionnaire data that you will first need to clean (a little).

Credit: [data source](https://archive.ics.uci.edu/ml/datasets/Student+Performance)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Load the dataset
students = pd.read_csv('student_grades.csv')
display(students.head())
students.shape

The dataset contains the following information:
- `male`: whether the student's sex is M or F (0, 1)
- `age`: the student's age (15-22)
- `address`: whether the student lives in an urban (U) or rural (R) setting
- `parents_together`: whether the parents live together or not
- `parents_avg_edu`: the rounded average of parental education
    - 0: none
    - 1: primary education (4th grade)
    - 2: 5th to 9th grade
    - 3: secondary education
    - 4: higher education
- `classes_failed`: the number of classes failed (n if 1 <= n < 3, else 4)
- `school_support`: extra educational support (yes, no)
- `absences`: number of school absences (0-93)
- `grade`: The student's grade (0-20)

In [ ]:
# Q1: What is the type of data of each variable?
"""
Numerical discrete: ...
Numerical continuous: ...
Categorical ordinal: ...
Categorical nominal: ...
"""

Alright, now we can start with the EDA and data cleaning.

First, we notice that `address` and `school_support` are binary variables but do not have (1, 0) as their values. Do the following:
- Replace the `address` column with a binary column named `rural` that takes the value 1 if a students comes from a rural setting and 0 otherwise.
- In the `school_support` column, replace `yes` with `1` and `no` with `0`.

Make sure to double-check that the datatypes (of the entire dataframe) are correct once you are done. Is there any column that is not yet in the datatype it should be?

In [ ]:
# Q2: Treat the binary columns that need to be formatted
# address to rural
students['rural'] = ...
students.drop('address', axis=1, inplace=True)

# school_support value transformation
students['school_support'] = ...

# Print the datatypes
print(students.dtypes)

# Display the dataframe
students.head()

Now, look if there are missing values. If there are, replace them with the median value of the column.

In [ ]:
# Q3: Treat the missing values
# Lookup the missing values
...

# Input the median if needed
...

Now, look up summary statistics for all variables and plot a histogram for them. Do you find any outliers? If so, remove the observation they occur in?

Hint: you should find exactly 1 outlier and it is easy to spot as it is outside of the possible values.

In [ ]:
# Q4: Remove outliers
...

# Plot all variables
...

# Remove the outlier
...

We would like to log-transform the dependent variable (i.e., `grade`). Since there are some students who received a 0, you will need to replace those values with an infinitely small positive number. Also, make sure the rename the `grade` column to `ln_grade`

In [ ]:
# Q5: Log-transform the dependent variable
# Replace 0 with 0.0000001 in grades
...

# Log-transform grade
...

# Rename grade to ln_grade
...

students

Finally, create two dummy variables out of `absences`:
- `some_abs`: 1 if `absences` is between 10 and 20 (included), 0 otherwise
- `many_abs`: 1 if `absences` is larger than 20, 0 otherwise

Think about why we do not create a third dummy variable for students with less than 10 absences.

Note: you are essentially asked to create two binary variables

In [ ]:
# Q6: Create a dummy variable from 'absences'
# Create the some_abs column
students['some_abs'] = np.where(
    (students['absences'] ...) & (students['absences'] ...),
    1,
    0
)

# Create the many_abs column
...

# Delete the absences column
...

students.head()

Alright, now we have a clean dataset that is ready for the actual machine learning part.

First, use the `statsmodels` library to compute a simple regression. In this case, we want you to regress `ln_grade` on all other variables without splitting the dataset. Hence, it should be just the way you ran regressions in your core statistics classes.

Interpret the results. Which coefficients are statistically significant? How do you interpret the coefficients?

In [ ]:
# Q7: Regression
# Regress the log-grade on all variables and a constant
regression = ...
print(regression.summary())

Finally, we will try to determine which two features predict the grade the best. To do so, we will train the model iteratively over every pair of features and retain the ones with the best test MSE.

Simply complete the code below to determine which model is the best.

Note: This is an oversimplified version of determining the best model. In reality, there is no indication that 2 is the correct number of features and you might want to use more metrics than just the MSE to determine the best model. This is just so that you can get a first exposure to what this process looks like and gain intuition.

In [ ]:
# Q8: Find the best 2-feature model
# Array to save the test errors
test_mse = pd.DataFrame(columns=['features', 'test_mse'])

# Feature pairs to evaluate
features = list(students.columns)
features.remove('ln_grade')
feature_pairs = []
for i, feature_1 in enumerate(features):
    for feature_2 in features[i+1:]:
        feature_pairs.append([feature_1, feature_2])

# Define the target
y = students['ln_grade']

# Iterate over the number of features
for feature_pair in feature_pairs:
    # Select features
    X_temp = students[feature_pair]

    # Split the dat set
    X_train, X_test, y_train, y_test = train_test_split(
        X_temp,
        y,
        test_size=0.2,
        random_state=12
    )

    # Create the linear model
    LR = ...

    # Fit the linear model
    ...
    
    # Compute the MSEs
    obs = pd.DataFrame({
        'features': [', '.join(feature_pair)],
        'test_mse': [mean_squared_error(y_test, LR.predict(X_test))]
    })

    test_mse = pd.concat([test_mse, obs], axis=0, ignore_index=True)

# Sort by MSEs
test_mse.sort_values(by='test_mse', axis=0, ascending=True, inplace=True)
test_mse